# Operazioni CRUD con Cassandra

Cassandra si installa da [Apache.org](https://cassandra.apache.org/download/) dove sono riportate le istruzioni per installazione di pacchetti Linux o da codice sorgente tramite repository [git](https://github.com/apache/cassandra). Per Windows vanno scaricati gli archivi binari.

E' necessario avere installato un JDK 8 o 11 che dev'essere raggiungibile dal proprio PATH.

Su MacOS utilizzeremo direttamente  `brew`. Una volta aggiunto il percorso ```/path/to/cassandra/bin``` alla propria variabile di ambiente ```PATH``` si può accedere all'istanza del server digitando semplicemente ```cassandra```. 

In ogni distribuzione, dopo l'avvio del servizio si potrà accedere alla shell interattiva digitando ```cqlsh``` che richiede Python3.6 o superiore.

## Comandi di shell

Una volta connessi al server attraverso ```cqlsh``` si possono usare diversi comandi di shell:

```
DESCRIBE               -- descrive lo stato generale del sistema piuttosto che singoli keyspace, tabelle o colonne
SHOW                   -- mostra informazioni su Cassandra e sul host cui si è connessi
CLEAR                  -- pulisce la shell
CONSISTENCY            -- riporta o imposta il livello di consistenza
SOURCE 'file'          -- esegue un file di comandi CQL
COPY table TO 'file'   -- copia una tabella su un file csv
COPY table FROM 'file' -- importa una tabella esistente da un file csv
EXIT                   -- uscita
LOGIN username [pass]  -- login come utente registrato
```

## Comandi CQL

Il corrispondente di un database è un keyspace che va creato ed utilizzato prima di creare le column family.

```
CREATE  KEYSPACE [IF NOT EXISTS] keyspace_name 
   WITH REPLICATION = { 
      'class' : 'SimpleStrategy', 'replication_factor' : N 
     | 'class' : 'NetworkTopologyStrategy', 
       'dc1_name' : N [, ...] 
   }
   [AND DURABLE_WRITES =  true|false] ;
   
   
USE keyspace_name;
```

La creazione di una column family richiede la specifica del [tipo CQL](https://docs.datastax.com/en/cql/3.3/cql/cql_reference/cql_data_types_c.html) delle varie colonne.

```
CREATE TABLE [IF NOT EXISTS] [keyspace_name.]table_name ( 
   column_definition [, ...]
   PRIMARY KEY (column_name [, column_name ...])
[WITH table_options
   | CLUSTERING ORDER BY (clustering_column_name order])
   | ID = 'table_hash_tag'
   | COMPACT STORAGE]
```

Creiamo il keyspace del Titanic con la tabella passeggeri.

```sql
CREATE KEYSPACE titanic WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 3};

USE titanic;

CREATE TABLE passengers (PassengerId int PRIMARY KEY, Survived int, Pclass int , 
   Name text , Sex text , Age float , SibSp int, Parch int, Ticket text, 
   Fare float, Cabin text, Embarked text);

COPY passengers (passengerid , survived , pclass , name , sex , age , sibsp , parch , ticket , fare , cabin , embarked ) 
FROM './Data/titanic.csv'  
WITH SKIPROWS = 1;

```

Le query sulle tabelle si effettuano con il costrutto ```SELECT``` e si possono creare indici su colonna per le clausole ```WHERE```.

```
CREATE INDEX IF NOT EXISTS index_name
ON keyspace_name.table_name ( KEYS ( column_name ) )

SELECT * | select_expression | DISTINCT partition 
FROM [keyspace_name.] table_name 
[WHERE partition_value
   [AND clustering_filters 
   [AND static_filters]]] 
[ORDER BY PK_column_name ASC|DESC] 
[LIMIT N]
[ALLOW FILTERING]
```
Selezioniamo tutte le righe.
Contiamo i passeggeri di terza classe.
Selezioniamo nome e cabina dei passeggeri di prima e seconda classe.

```sql
CREATE INDEX IF NOT EXISTS class ON passengers (pclass);

SELECT COUNT(*) FROM passengers WHERE pclass = 3;

SELECT name, cabin FROM passengers WHERE pclass < 3 ALLOW FILTERING ;
```

L'alterazione si esegue con ```ALTER```.

```
ALTER TABLE [keyspace_name.] table_name 
[ALTER column_name TYPE cql_type]
[ADD (column_definition_list)]
[DROP column_list | COMPACT STORAGE ]
[RENAME column_name TO column_name]
[WITH table_properties];

ALTER  KEYSPACE keyspace_name 
   WITH REPLICATION = { 
      'class' : 'SimpleStrategy', 'replication_factor' : N  
     | 'class' : 'NetworkTopologyStrategy', 'dc1_name' : N [, ...] 
   }
   [AND DURABLE_WRITES =  true|false] ;
```
Eliminiamo una colonna:

```sql
ALTER TABLE passengers DROP sibsp;
```

L'inserimento dati si esegue con ```INSERT```.

```
INSERT INTO [keyspace_name.] table_name (column_list) 
VALUES (column_values) 
[IF NOT EXISTS] 
[USING TTL seconds | TIMESTAMP epoch_in_microseconds]
```
La cancellazione si esegue con ```DELETE```.

```
DELETE [column_name (term)][, ...]
FROM [keyspace_name.] table_name 
[USING TIMESTAMP timestamp_value]
WHERE PK_column_conditions 
[IF EXISTS | IF static_column_conditions]
```

Inseriamo un nuovo passeggero e poi rimuoviamolo.

```sql
INSERT INTO passengers (passengerid , age , cabin , embarked , fare , name , parch , pclass , sex , survived , ticket ) 
VALUES ( 892, 45.0, 'Z76', 'S', 456.789, 'John Smith', 2, 1, 'male', 1, 'TicketOne');

DELETE name, sex FROM passengers WHERE passengerid = 891;

SELECT * FROM passengers WHERE passengerid = 891; # i valori cancellati sono nulli

DELETE FROM passengers WHERE passengerid = 891;
```
L'aggiornamento si effettua con ```UPDATE```.

```
UPDATE [keyspace_name.] table_name
[USING TTL time_value | USING TIMESTAMP timestamp_value]
SET assignment [, assignment] . . . 
WHERE row_specification
[IF EXISTS | IF condition [AND condition] . . .] ;
```
Modifichiamo il tipo di biglietto del passeggero n. 892.

```sql
UPDATE passengers SET ticket = 'VivaTicket' WHERE passengerid = 892;
```


La rimozione si esegue con il comando ```DROP```.

```
DROP TABLE [IF EXISTS] keyspace_name.table_name

DROP KEYSPACE [IF EXISTS] keyspace_name

DROP INDEX [IF EXISTS] [keyspace.]index_name

```
Rimuoviamo la tabella dei passeggeri

```sql
DROP TABLE passengers;
```



## Aggregazione

E' possibile creare funzioni e aggregazioni definite dall'utente usando codice Java o Javascript, ma ci sono già disponibili ```MAX, MIN, COUNT, AVG```. Per gestire al meglio le query complesse con aggregazioni e raggruppamenti bisogna creare le giuste chiavi primarie che creano la cosiddetta _partition key_. 

Al fine di rendere efficienti le query di ricerca e aggregazione, è bene creare una tabella con chiave primaria composita con una struttura che ricalchi il seguente esempio.

```sql
CREATE TABLE example (
    partitionKey1 text,
    partitionKey2 text,
    clusterKey1 text,
    clusterKey2 text,
    normalField1 text,
    normalField2 text,
    PRIMARY KEY (
        (partitionKey1, partitionKey2),
        clusterKey1, clusterKey2
        )
    );
```

Questa chiave è composita e contiene, nella prima parte, tutti e soli i campi che serviranno nelle clausole ```WHERE``` e ```GROUP BY``` della query (unici consentiti per altro).  
I campi che fanno parte di quella porzione della chiave primaria che è utilizzata per il clustering delle righe, sono quelli che effettivamente distinguono le singole righe e quindi consentono di raggruppare più righe distinte per lo stesso valore dei campi della partition key. Tali righe costituiscono la base per eseguire efficientemente le aggregazioni.

Ricreiamo la struttura della tabella dei passeggeri e calcoliamo il numero e l'età media dei passeggeri sopravvissuti per classe e per genere.

```sql
CREATE TABLE passengers (
    survived int,
    class int,
    sex text,
    id int,
    age float,
    cabin text,
    embarked text,
    fare float,
    name text,
    parch int,
    sibsp int,
    ticket text,
    PRIMARY KEY ((survived, class, sex), id)
) WITH CLUSTERING ORDER BY (id ASC);

COPY passengers (id , survived , class , name , sex , age , sibsp , parch , ticket , fare , cabin , embarked ) FROM './src/github repositories/Big-Data/Data/titanic.csv' WITH SKIPROWS = 1;

SELECT DISTINCT survived, class, sex FROM passengers ;

SELECT COUNT(*) AS num_of_pass, AVG(age) as average_age, sex, class FROM passengers WHERE survived=1 AND class < 4 GROUP BY class, sex ALLOW FILTERING;
```

## Utilizzo del driver Python per Cassandra

Il driver può essere installato usando ```conda```:

```sh
$ conda install -c conda-forge cassandra-driver
```

ovvero `pip`:

```sh
$ pip install cassandra-driver
```

In [1]:
"""
import cassandra.cluster as ccl
import cassandra.policies as pol

# connessione ad un cluster composto da tre nodi,
# in ascolto sulla porta 9042, con una determinata
# politica di bilanciamento del carico di tipo round-robin

cluster = ccl.Cluster(
    ['10.1.1.3', '10.1.1.4', '10.1.1.5'],
    load_balancing_policy=pol.DCAwareRoundRobinPolicy(local_dc='US_EAST'),
    port=9042)
""" 
import cassandra.cluster as ccl

# connessione al cluster di default
cluster = ccl.Cluster()
session = cluster.connect()


In [2]:
# creiamo ed usiamo il keyspace
session.execute("CREATE KEYSPACE titanic WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 3}")
session.set_keyspace('titanic')

# session.execute('USE titanic')

In [3]:
# Creiamo la tabella 
query = "CREATE TABLE titanic.passengers (\
    survived int,\
    class int,\
    sex text,\
    id int,\
    age float,\
    cabin text,\
    embarked text,\
    fare float,\
    name text,\
    parch int,\
    sibsp int,\
    ticket text,\
    PRIMARY KEY ((survived, class, sex), id)\
) WITH CLUSTERING ORDER BY (id ASC)"

session.execute(query)

In [5]:
# importiamo i dati dal dataset csv
# COPY è un comando di cqlsh e non un comando CQL
# quindi lo eseguiremo invocando cqlsh e passando
# il comando da eseguire
import os

os.system("cqlsh -k titanic -e \"COPY passengers (id , survived , class , name , sex , age , sibsp , parch , ticket , fare , cabin , embarked ) FROM '~/data/titanic.csv' WITH SKIPROWS = 1\"")

Using 7 child processes

Starting copy of titanic.passengers with columns [id, survived, class, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked].
Processed: 891 rows; Rate:    1276 rows/s; Avg. rate:    1976 rows/s
891 rows imported from 1 files in 0.451 seconds (1 skipped).


0

In [6]:
# Eseguiamo una query di insert
session.execute("""
INSERT INTO titanic.passengers (
    survived,
    class,
    sex,
    id, 
    age,
    cabin,
    embarked,
    fare,
    name,
    parch,
    sibsp,
    ticket) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""",
                ( 1, 1, 'male', 892, 45.0, 'Z76', 'S', 456.789, 'John Smith', 2, 1, 'TicketOne'))

In [7]:
# Eseguiamo una query di selezione
rows = session.execute('SELECT name, age, cabin, fare FROM titanic.passengers WHERE id > 880 ALLOW FILTERING')

for row in rows:
    print(row.name, row.age, row.cabin, row.fare, sep='\n', end='\n-------------\n')



Dahlberg, Miss. Gerda Ulrika
22.0
None
10.51669979095459
-------------
Rice, Mrs. William (Margaret Norton)
39.0
None
29.125
-------------
Johnston, Miss. Catherine Helen "Carrie"""
None
None
23.450000762939453
-------------
Shelley, Mrs. William (Imanita Parrish Hall)
25.0
None
26.0
-------------
Banfield, Mr. Frederick James
28.0
None
10.5
-------------
Montvila, Rev. Juozas
27.0
None
13.0
-------------
Behr, Mr. Karl Howell
26.0
C148
30.0
-------------
John Smith
45.0
Z76
456.78900146484375
-------------
Markun, Mr. Johann
33.0
None
7.8958001136779785
-------------
Sutehall, Mr. Henry Jr
25.0
None
7.050000190734863
-------------
Dooley, Mr. Patrick
32.0
None
7.75
-------------
Graham, Miss. Margaret Edith
19.0
B42
30.0
-------------


In [8]:
# Eseguiamo una query invocando esplicitamente la classe SimpleStatement che
# definisce i comandi CQL nella API del driver
import cassandra

query = cassandra.query.SimpleStatement(
    "INSERT INTO titanic.passengers (survived, class, sex, id, name, age) VALUES (%s, %s, %s, %s, %s, %s)",
    consistency_level=cassandra.ConsistencyLevel.ONE) # pleonastico perché c'è un solo nodo nel nostro cluster

session.execute(query, (0, 3, 'female', 893, 'Mrs. Melania De Avilland', 42))


In [9]:
# E' possibile preparare le query ed eseguirle parametricamente
# con la classe PreparedStatement per la quale si può impostare il livello di consistenza

survived_per_class_stmt = session.prepare("SELECT * FROM titanic.passengers WHERE survived = 1 AND  class = ? ALLOW FILTERING")
survived_per_class_stmt.consistency_level = cassandra.ConsistencyLevel.ONE

survived = []
for pclass in [1,2,3]:
    passengers = session.execute(survived_per_class_stmt, [pclass])
    survived.append(passengers)


In [10]:
for passenger in survived[0]:
    print(passenger.name,passenger.sex,sep='\n',end='\n---------------\n')

Sloper, Mr. William Thompson
male
---------------
Woolner, Mr. Hugh
male
---------------
Greenfield, Mr. William Bertram
male
---------------
Romaine, Mr. Charles Hallace ("Mr C Rolmane"")"
male
---------------
Blank, Mr. Henry
male
---------------
Hoyt, Mr. Frederick Maxfield
male
---------------
Beckwith, Mr. Richard Leonard
male
---------------
Saalfeld, Mr. Adolphe
male
---------------
Allison, Master. Hudson Trevor
male
---------------
Harder, Mr. George Achilles
male
---------------
Carter, Mr. William Ernest
male
---------------
Bjornstrom-Steffansson, Mr. Mauritz Hakan
male
---------------
Dodge, Master. Washington
male
---------------
Seward, Mr. Frederic Kimber
male
---------------
Peuchen, Major. Arthur Godfrey
male
---------------
Goldenberg, Mr. Samuel L
male
---------------
Anderson, Mr. Harry
male
---------------
Bishop, Mr. Dickinson H
male
---------------
Bradley, Mr. George ("George Arthur Brayton"")"
male
---------------
McGough, Mr. James Robert
male
---------------

### Esecuzione asincrona

L'esecuzione asincrona consente di inviare la query continuare l'esecuzione e poi raccogliere i risultati. Si possono anche definire delle callback per la gestione del successo e del fallimento della query.

Nel seguente esempio si gestiscono delle query concorrenti.

```python
# Questa è l'eccezione di timeout delle query
from cassandra import ReadTimeout

# lista dei risultati futuri
futures = []
query = "SELECT * FROM users WHERE user_id=%s"
for user_id in ids_to_fetch:
    futures.append(session.execute_async(query, [user_id])

# le query vengono eseguite in modo asincrono e concorrente
# e questo codice attende che siano finite
for future in futures:
    rows = future.result()
    print rows[0].name

```
Esempio di definizione delle funzioni di callback.

```python

# definiamo le due callback
def handle_success(rows):
    user = rows[0]
    try:
        process_user(user.name, user.age, user.id)
    except Exception:
        log.error("Failed to process user %s", user.id)

def handle_error(exception):
    log.error("Failed to fetch user info: %s", exception)

# lanciamo la query asincrona e agganciamo le due funzioni di callback
future = session.execute_async(query)
future.add_callbacks(handle_success, handle_error)
```

### Utilizzo degli Object Mapper

In [11]:
# Effettuiamo il drop della tabella

session.execute('DROP TABLE titanic.passengers')

In [12]:
# La API per generare gli Object Mapper
# si trova nel modulo cassandra.cqlengine
import cassandra
from cassandra.cqlengine import columns
from cassandra.cqlengine import connection
from datetime import datetime
from cassandra.cqlengine.management import sync_table
from cassandra.cqlengine.models import Model


# setup della connessione
connection.setup(['127.0.0.1'],'titanic',consistency=cassandra.ConsistencyLevel.ONE)

In [13]:
# Creazione del modello dei dati,
# cioè la tabella, come classe derivata da Model

class Passengers(Model):
    survived = columns.Integer(primary_key=True,partition_key=True)
    pclass = columns.Integer(primary_key=True,partition_key=True)
    sex = columns.Text(primary_key=True,partition_key=True)
    pid = columns.Integer(primary_key=True, clustering_order='asc') 
    age = columns.Float()
    cabin = columns.Text()
    embarked = columns.Text()
    fare = columns.Float()
    name = columns.Text()
    parch = columns.Integer()
    sibsp = columns.Integer()
    ticket = columns.Text() 


In [14]:
# Viene create una tabella dal nome uguale al modello
# all'interno del keyspace di default della connessione

sync_table(Passengers)

/home/rpirrone/virtualenvs/hadoop/lib/python3.10/site-packages/cassandra/cqlengine/management.py:545: UserWarning: CQLENG_ALLOW_SCHEMA_MANAGEMENT environment variable is not set. Future versions of this package will require this variable to enable management functions.
  warnings.warn(msg)


In [16]:
# importiamo i dati richiamando cqlsh da sistema
import os

os.system("cqlsh -k titanic -e \"COPY passengers (pid , survived , pclass , name , sex , age , sibsp , parch , ticket , fare , cabin , embarked ) FROM '~/data/titanic.csv' WITH SKIPROWS = 1\"")

Using 7 child processes

Starting copy of titanic.passengers with columns [pid, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked].
Processed: 891 rows; Rate:    1305 rows/s; Avg. rate:    2008 rows/s
891 rows imported from 1 files in 0.444 seconds (1 skipped).


0

In [17]:
# eseguiamo operazioni di insert
Passengers.create(survived=1,pclass=1,sex='male',pid=892,age=45.0,cabin='Z76',embarked='S',fare=123.456,name='John Smith',parch=2,sibsp=1,ticket='TiketOne')

Passengers(survived=1, pclass=1, sex='male', pid=892, age=45.0, cabin='Z76', embarked='S', fare=123.456, name='John Smith', parch=2, sibsp=1, ticket='TiketOne')

In [18]:
# manipoliamo gli oggetti della tabella attraverso la classe model
# che consente di fare query e update
Passengers.objects.count()

892

In [19]:
# l'operazione di update dev'essere fatta selezionando l'intera chiave primaria

Passengers.objects(survived=1,pclass=1,sex='male',pid=892).update(ticket='VivaTicket')

In [20]:
q = Passengers.objects.filter(survived=1,pclass__in=[1,2],sex='female')

for passenger in q:
    print(passenger.name,passenger.age,sep='\n',end='\n---------------\n')

Cumings, Mrs. John Bradley (Florence Briggs Thayer)
38.0
---------------
Futrelle, Mrs. Jacques Heath (Lily May Peel)
35.0
---------------
Bonnell, Miss. Elizabeth
58.0
---------------
Spencer, Mrs. William Augustus (Marie Eugenie)
None
---------------
Harper, Mrs. Henry Sleeper (Myna Haxtun)
49.0
---------------
Icard, Miss. Amelie
38.0
---------------
Fortune, Miss. Mabel Helen
23.0
---------------
Newsom, Miss. Helen Monypeny
19.0
---------------
Pears, Mrs. Thomas (Edith Wearne)
22.0
---------------
Chibnall, Mrs. (Edith Martha Bowerman)
None
---------------
Brown, Mrs. James Joseph (Margaret Tobin)
44.0
---------------
Lurette, Miss. Elise
58.0
---------------
Newell, Miss. Madeleine
31.0
---------------
Bazzani, Miss. Albina
32.0
---------------
Harris, Mrs. Henry Birkhardt (Irene Wallach)
35.0
---------------
Thorne, Mrs. Gertrude Maybelle
None
---------------
Cherry, Miss. Gladys
30.0
---------------
Ward, Miss. Anna
35.0
---------------
Graham, Mrs. William Thompson (Edith Jun

In [21]:
# non funziona perché bisogna etichettare cabin come indice
# al momento della creazione della tabella

os.system("cqlsh -k titanic -e \"CREATE INDEX cabin_idx ON passengers(cabin)\"")

q = Passengers.objects.filter(survived=1,pclass__in=[1,2],sex='female',cabin__like='C%')

for passenger in q:
    print(passenger.name,passenger.age,sep='\n',end='\n---------------\n')

InvalidRequest: Error from server: code=2200 [Invalid query] message="LIKE restriction is only supported on properly indexed columns. cabin LIKE 'C%' is not valid."

In [22]:
# aggregazione con valori medi -- fuori dal driver
# tasso di sopravvivenza

import numpy as np
import pandas as pd


data_set=[]

females_1_class = Passengers.objects(survived=1,pclass=1,sex='female')
females_embarked_1_class = Passengers.objects.filter(pclass=1,sex='female')
males_1_class = Passengers.objects(survived=1,pclass=1,sex='male')
males_embarked_1_class = Passengers.objects.filter(pclass=1,sex='male')
females_2_class = Passengers.objects(survived=1,pclass=2,sex='female')
females_embarked_2_class = Passengers.objects.filter(pclass=2,sex='female')
males_2_class = Passengers.objects(survived=1,pclass=2,sex='male')
males_embarked_2_class = Passengers.objects.filter(pclass=2,sex='male')
females_3_class = Passengers.objects(survived=1,pclass=3,sex='female')
females_embarked_3_class = Passengers.objects.filter(pclass=3,sex='female')
males_3_class = Passengers.objects(survived=1,pclass=3,sex='male')
males_embarked_3_class = Passengers.objects.filter(pclass=3,sex='male')

def avg_age(query):
    
    age_array = np.array([item.age for item in query])
    
    return age_array[age_array != None].mean()

data_set = [\
    {'class': 1,\
    'female_%': females_1_class.count()/females_embarked_1_class.allow_filtering().count(),\
    'f_avg_age': avg_age(females_1_class),\
    'male_%': males_1_class.count()/males_embarked_1_class.allow_filtering().count(),\
    'm_avg_age': avg_age(males_1_class)},\
    {'class': 2,\
    'female_%': females_2_class.count()/females_embarked_2_class.allow_filtering().count(),\
    'f_avg_age': avg_age(females_2_class),\
    'male_%': males_2_class.count()/males_embarked_2_class.allow_filtering().count(),\
    'm_avg_age': avg_age(males_2_class)},\
    {'class': 3,\
    'female_%': females_3_class.count()/females_embarked_3_class.allow_filtering().count(),\
    'f_avg_age': avg_age(females_3_class),\
    'male_%': males_3_class.count()/males_embarked_3_class.allow_filtering().count(),\
    'm_avg_age': avg_age(males_3_class)}\
]        

dataframe = pd.DataFrame(data_set)

dataframe.set_index('class')

,female_%,f_avg_age,male_%,m_avg_age
class,,,,
1,0.968085,34.939024,0.373984,36.461463
2,0.921053,28.080882,0.157407,16.022000
3,0.500000,19.329787,0.135447,22.274211
